
<div style="line-height:2px;border: solid orange">
    <p>
    <p style="color:blue;font-family:arial;text-align:right;font-size:20"> Visión por Computador &nbsp;&nbsp;
    <p style="color:blue;font-family:arial;text-align:right;font-size:16"> Master en Ciencias de Datos e Ingeniería de Ordenadores   &nbsp;&nbsp;
  <p style="color:blue;font-family:arial;text-align:right;font-size:16"> Rosa Mª. Rodríguez Sánchez   &nbsp;&nbsp;
    <p style="color:blue;font-family:arial;text-align:right;font-size:10"> Dpto. Ciencias de la Computación e Inteligencia Artificial. &nbsp;&nbsp;  
    <p style="color:blue;font-family:arial;text-align:right;font-size:10"> ETSIIT. Universidad de Granada   &nbsp;&nbsp;
        <p>
</div>

# Práctica 5 (Parte II). Transformada de Fourier

### Introducción
Joseph Fourier fue un matemático del siglo 18 que descubrió que todas las funciones con formas onduladas se pueden representar mediante una suma de sinusoidales de diferentes frecuencias y con diferentes amplitudes. Este mecanismo es el principio de la Transformada de Fourier TF. 
Una imagen se puede ver como una señal 2d (funcion 2d) y por lo tanto es susceptible de describir una imagen como un conjunto de fuciones senos 2d. La utilidad de aplicar la TF es permitirnos identificar regiones de la imagen en que los valores de los pixeles cambian mucho, o regiones más constantes. Por lo tanto analizando la TF de una imagen podemos etiquetar esa región como de interés, o ruido, fondo, primer plano, etc.
Por lo tanto, haciendo un análisis de una imagen en el dominio de Fourier podemos realizar tareas como emborronamiento, detección de aristas, umbralización, analisis de texturas e incluso detectar si existe emborronamiento.

OpenCV tiene las funcion  **cv2.dft()** para obtener la TF de una imagen y la función **cv2.idft()** para a partir TF recuperar la imagen en el dominio espacial. 
En el siguiente código podemos ver para una imagen síntética su transformada de Fourier (espectro de la magnitud y fase).

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = np.zeros((400,400), dtype = "uint8") 
cv2.rectangle(img, (150,150), (250,250), (255,255,255), -1)


img_float32 = np.float32(img)

dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)
magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))
#True para ser angulos en grados o false en radianes
phase_spectrum = cv2.phase(dft_shift[:,:,1],dft_shift[:,:,0],True)


#Para obtener la imagen original
idft_shift = np.fft.ifftshift(dft_shift)
#consultar el significado de los flags 
#https://docs.opencv.org/4.x/d2/de8/group__core__array.html#gaf4dde112b483b38175621befedda1f1c
img_rec=cv2.idft(idft_shift, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT);
plt.figure(figsize=(8,8))
plt.subplot(131),plt.imshow(img, cmap = 'gray')
plt.title('Image Original'), plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude del Espectro'), plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(phase_spectrum, cmap = 'gray')
plt.title('Fase del Espectro'), plt.xticks([]), plt.yticks([])
plt.figure(figsize=(3,3))
plt.imshow(img_rec,'gray'),plt.title('Recuperada')

In [ ]:
#Aplicar sobre una imagen de grises
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('miki_pluto.jpg',0)

img_float32 = np.float32(img)

dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))
phase_spectrum = cv2.phase(dft_shift[:,:,1],dft_shift[:,:,0],True)
plt.figure(figsize=(8,8))
plt.subplot(131),plt.imshow(img, cmap = 'gray')
plt.title('Image Original'), plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude del Espectro'), plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(phase_spectrum, cmap = 'gray')
plt.title('Fase del Espectro'), plt.xticks([]), plt.yticks([])
plt.show()         

## Armónicos.
Vamos a visualizar ahora determinados armónicos. Para ello activamos determinadas frecuencias en el dominio de Fourier para luego obtener la transformada inversa de Fourier y visualizar en el dominio espacial.

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

#Opencv representa los complejos como un matriz con dos columnas
#una para la parte real y otra para la parte imaginaria
F=np.zeros((256,256,2));
#Componente verticales
FV=F.copy()
FV[0,5,0]=1
FV[0,5,1]=1
#Componente horizontales
FH=F.copy();
FH[5,0,0]=1;
FH[5,0,1]=1;
Fsuma =FV+FH
IFv= cv2.idft(FV, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
IFh= cv2.idft(FH, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
IFs= cv2.idft(Fsuma, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
plt.subplot(1,3,1),plt.imshow(IFv,'gray'),plt.title('Vertical')
plt.subplot(1,3,2),plt.imshow(IFh,'gray'),plt.title('Horizontal')  
plt.subplot(1,3,3),plt.imshow(IFs,'gray'),plt.title('Suma')  


Podemos seleccionar armónicos que estén dentro de un area. Por ejemplo dentro de un disco como se muestra a continuación.

In [ ]:
#Cambiar para obtener Armonicos que en un disco de radio 4 centrado en la %frecuencia 200 220
import numpy as np
import cv2
from matplotlib import pyplot as plt

F=np.zeros((400,400,2))
dist=np.zeros((400,400));
mi=400/2;
mj=400/2;

i_ind=np.arange(400)
j_ind=np.arange(400)
xx, yy = np.meshgrid(j_ind, i_ind, sparse=True)
dist=np.hypot(yy-mi,xx-mj);
radius=2

F[:,:,0]=np.where(dist<=radius,1.0,0.0);
F[:,:,1]=np.where(dist<=radius,1.0,0.0);

#Obtenemos los armonicos en el dominio espacial
iF = np.fft.ifftshift(F)
IF= cv2.idft(iF, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
plt.figure(figsize=(4,4 ));
plt.contourf(j_ind,i_ind,dist),plt.title("Distancias")
plt.figure(figsize=(8,8));
plt.subplot(1,2,1),plt.imshow(F[:,:,0],'gray'),plt.title("Disco")
plt.subplot(1,2,2),plt.imshow(IF,'gray'),plt.title("Armonicos")

print(np.unique(iF[:,:,0]))

In [ ]:

import numpy as np
import cv2
from matplotlib import pyplot as plt

#Funcion para obtener el arco tangente de x,y
def ATN(y, x):
    atn = np.arctan2(y, x)
    atn[atn < 0] += 2*np.pi
    return atn



F=np.zeros((400,400,2))
dist=np.zeros((400,400));
angulos=np.zeros((400,400));
mi=400/2;
mj=400/2;
i_ind=np.arange(400)
j_ind=np.arange(400)
xx, yy = np.meshgrid(j_ind, i_ind, sparse=True)

dist=np.hypot(yy-mi,xx-mj);
#obtenemos una imagen con valores de angulos en cada pixel
angulos = ATN(yy-mi,xx-mj)

radius=23
#seleccionamos todo el espectro comprendido entre 0-95 grados
min_ori=0*np.pi/180
max_ori=95*np.pi/180
#calculamos la mascara 
mask_sector=(angulos>=min_ori)*1
mask_sector=cv2.bitwise_and(mask_sector,(angulos<max_ori)*1)
#calculamos el circulo de radio radius
mask_disc = (dist<=radius)*1
#seleccionamos el sector del disco comprendido entre 0-95graods
mask = cv2.bitwise_and(mask_sector,mask_disc)
F[:,:,0]=np.where(mask==1 ,1,0);
F[:,:,1]=np.where(mask==1 ,1,0);

#Obtenemos los armonicos en el dominio espacial
iF = np.fft.ifftshift(F)
IF= cv2.idft(iF, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
plt.figure(figsize=(4,4));
plt.contourf(j_ind,i_ind,dist),plt.title("Distancias")
plt.figure(figsize=(8,8));
plt.subplot(1,2,1),plt.imshow(F[:,:,0],'gray'),plt.title("Disco")
plt.subplot(1,2,2),plt.imshow(IF,'gray'),plt.title("Armonicos")


### Filtrado Paso Bajo (LPF)
Podemos obtener mediante ese mecanismo de selección de armónicos que información afecta a una imagen, simplemente multiplicando en el dominio de fourier la imagen y la selección (Filtro). Si eliminamos las frecuencias altas del contenido frecuencial de una image estamos aplicando un filtro paso bajo. Los tres tipos de filtros LPF que vamos a ver son:
* Filtro Paso Bajo ideal
* Filtros Paso bajo gaussiano
* Filtros Paso bajo Butterworth
El efecto de estos filtros es emborronar la imagen. Puede ser interesante para una imagen con ruido aplicar un filtro paso bajo como preprocesamiento. 



In [ ]:
#Filtro Paso Bajo Ideal
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('miki_pluto.jpg',0)

img_float32 =np.float32(img)
#obtenemos TF de la imagen de entrada
dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

F=np.zeros((img.shape[0],img.shape[1],2))
dist=np.zeros(F.shape[:2]);
mi=img.shape[0]/2;
mj=img.shape[1]/2;
i_ind=np.arange(img.shape[0])
j_ind=np.arange(img.shape[1])
xx, yy = np.meshgrid(j_ind, i_ind, sparse=True)
dist=np.hypot(yy-mi,xx-mj);
radius=30
F[:,:,0]=np.where(dist<=radius,1,0);
F[:,:,1]=np.where(dist<=radius,1,0);

#Multiplicamos con la selección F obtenida 

dft_sel=dft_shift * F #multiplicacion punto a punto
#Obtenemos la iTF a ver que selecciona
iF = np.fft.ifftshift(dft_sel)
IF= cv2.idft(iF, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
#IF= cv2.idft(iF); 
#IF = cv2.magnitude(IF[:,:,0],IF[:,:,1])

plt.figure(figsize=(12,12));
plt.subplot(1,3,1),plt.imshow(img,'gray'),plt.title("Original")
plt.subplot(1,3,2),plt.imshow(F[:,:,0],'gray'),plt.title("Filtro Ideal Paso Bajo")
plt.subplot(1,3,3),plt.imshow(IF,'gray'),plt.title("Imagen Seleccion")


Con el filtro paso bajo ideal surge un efecto no deseado en la imagen de salida que es el anillamiento. Veamos en el siguiente ejemplo el filtro paso bajo con una gaussiana

In [ ]:

#Filtro Paso Bajo gaussiano
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('miki_pluto.jpg',0)

img_float32 =np.float32(img)
#obtenemos TF de la imagen de entrada
dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

F=np.zeros((img.shape[0],img.shape[1],2))
dist=np.zeros(F.shape[:2]);
mi=img.shape[0]/2;
mj=img.shape[1]/2;
i_ind=np.arange(img.shape[0])
j_ind=np.arange(img.shape[1])
xx, yy = np.meshgrid(j_ind, i_ind, sparse=True)
dist=np.hypot(yy-mi,xx-mj);

sigma =30;

F[:,:,0]=np.exp(-(dist*dist)/(2*(sigma*sigma))); 
F[:,:,1]=np.exp(-(dist**2)/(2*(sigma*sigma))); 

#Multiplicamos con la selección F obtenida 

dft_sel=dft_shift * F #multiplicacion punto a punto
#Obtenemos la iTF a ver que selecciona
iF = np.fft.ifftshift(dft_sel)
IF= cv2.idft(iF, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 


plt.figure(figsize=(12,12));
plt.subplot(1,3,1),plt.imshow(img,'gray'),plt.title("Original")
plt.subplot(1,3,2),plt.imshow(F[:,:,0],'gray'),plt.title("Filtro  Paso Bajo Gaussiano")
plt.subplot(1,3,3),plt.imshow(IF,'gray'),plt.title("Imagen Seleccion")





A continuación veamos el filtro Butterworth.

In [ ]:

#Filtro Paso Butterworth
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('miki_pluto.jpg',0)

img_float32 =np.float32(img)
#obtenemos TF de la imagen de entrada
dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

F=np.zeros((img.shape[0],img.shape[1],2))
dist=np.zeros(F.shape[:2]);
mi=img.shape[0]/2;
mj=img.shape[1]/2;
i_ind=np.arange(img.shape[0])
j_ind=np.arange(img.shape[1])
xx, yy = np.meshgrid(j_ind, i_ind, sparse=True)
dist=np.hypot(yy-mi,xx-mj);

D0=35; n=2

F[:,:,0]=1/(1+(dist/D0)**(2*n));
F[:,:,1]=1/(1+(dist/D0)**(2*n));
#Multiplicamos con la selección F obtenida 

dft_sel=dft_shift * F #multiplicacion punto a punto
#Obtenemos la iTF a ver que selecciona
iF = np.fft.ifftshift(dft_sel)
IF= cv2.idft(iF, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
#IF= cv2.idft(iF); 
#IF = cv2.magnitude(IF[:,:,0],IF[:,:,1])

plt.figure(figsize=(12,12));
plt.subplot(1,3,1),plt.imshow(img,'gray'),plt.title("Original")
plt.subplot(1,3,2),plt.imshow(F[:,:,0],'gray'),plt.title("Filtro  Paso Bajo Butterworth")
plt.subplot(1,3,3),plt.imshow(IF,'gray'),plt.title("Imagen Seleccion")





### Filtrado Paso Alto (HPF)
Si eliminamos las frecuencias bajas  del contenido frecuencial de una imagen estamos aplicando un filtro paso bajo. Los tres tipos de filtro HPF que vamos a ver son:
* Filtro Paso Alto ideal
* Filtros Paso Alto gaussiano
* Filtros Paso Alto Butterworth
El efecto de estos filtros es un efecto frontera en la imagen.


In [ ]:
#Filtro Paso Alto Ideal
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('miki_pluto.jpg',0)

img_float32 =np.float32(img)
#obtenemos TF de la imagen de entrada
dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

F=np.zeros((img.shape[0],img.shape[1],2))
dist=np.zeros(F.shape[:2]);
mi=img.shape[0]/2;
mj=img.shape[1]/2;
i_ind=np.arange(img.shape[0])
j_ind=np.arange(img.shape[1])
xx, yy = np.meshgrid(j_ind, i_ind, sparse=True)
dist=np.hypot(yy-mi,xx-mj);
radius=30
F[:,:,0]=np.where(dist>radius,1,0);
F[:,:,1]=np.where(dist>radius,1,0);

#Multiplicamos con la selección F obtenida 

dft_sel=dft_shift * F #multiplicacion punto a punto
#Obtenemos la iTF a ver que selecciona
iF = np.fft.ifftshift(dft_sel)
IF= cv2.idft(iF, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
#IF= cv2.idft(iF); 
#IF = cv2.magnitude(IF[:,:,0],IF[:,:,1])

plt.figure(figsize=(12,12));
plt.subplot(1,3,1),plt.imshow(img,'gray'),plt.title("Original")
plt.subplot(1,3,2),plt.imshow(F[:,:,0],'gray'),plt.title("Filtro Ideal Paso Alto")
plt.subplot(1,3,3),plt.imshow(IF,'gray'),plt.title("Imagen Seleccion")


Con el filtro paso alto ideal surge un efecto no deseado en la imagen de salida que es como una doble frontera. Veamos en el siguiente ejemplo el filtro paso alto con una gaussiana

In [ ]:

#Filtro Paso Alto gaussiano
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('miki_pluto.jpg',0)

img_float32 =np.float32(img)
#obtenemos TF de la imagen de entrada
dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

F=np.zeros((img.shape[0],img.shape[1],2))
dist=np.zeros(F.shape[:2]);
mi=img.shape[0]/2;
mj=img.shape[1]/2;
i_ind=np.arange(img.shape[0])
j_ind=np.arange(img.shape[1])
xx, yy = np.meshgrid(j_ind, i_ind, sparse=True)
dist=np.hypot(yy-mi,xx-mj);

sigma =30;

F[:,:,0]=1-np.exp(-(dist*dist)/(2*(sigma*sigma))); 
F[:,:,1]=1-np.exp(-(dist**2)/(2*(sigma*sigma))); 
#Multiplicamos con la selección F obtenida 

dft_sel=dft_shift * F #multiplicacion punto a punto
#Obtenemos la iTF a ver que selecciona
iF = np.fft.ifftshift(dft_sel)
IF= cv2.idft(iF, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
#IF= cv2.idft(iF); 
#IF = cv2.magnitude(IF[:,:,0],IF[:,:,1])

plt.figure(figsize=(12,12));
plt.subplot(1,3,1),plt.imshow(img,'gray'),plt.title("Original")
plt.subplot(1,3,2),plt.imshow(F[:,:,0],'gray'),plt.title("Filtro  Paso Alto Gaussiano")
plt.subplot(1,3,3),plt.imshow(IF,'gray'),plt.title("Imagen Seleccion")





A continuación veamos el filtro Butterworth.

In [ ]:

#Filtro Paso Alto Butterworth
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('miki_pluto.jpg',0)

img_float32 =np.float32(img)
#obtenemos TF de la imagen de entrada
dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

F=np.zeros((img.shape[0],img.shape[1],2))
dist=np.zeros(F.shape[:2]);
mi=img.shape[0]/2;
mj=img.shape[1]/2;
i_ind=np.arange(img.shape[0])
j_ind=np.arange(img.shape[1])
xx, yy = np.meshgrid(j_ind, i_ind, sparse=True)
dist=np.hypot(yy-mi,xx-mj);

D0=35; n=2

F[:,:,0]=1/(1+(D0/dist)**(2*n));
F[:,:,1]=1/(1+(D0/dist)**(2*n));
#Multiplicamos con la selección F obtenida 

dft_sel=dft_shift * F #multiplicacion punto a punto
#Obtenemos la iTF a ver que selecciona
iF = np.fft.ifftshift(dft_sel)
IF= cv2.idft(iF, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT); 
#IF= cv2.idft(iF); 
#IF = cv2.magnitude(IF[:,:,0],IF[:,:,1])

plt.figure(figsize=(12,12));
plt.subplot(1,3,1),plt.imshow(img,'gray'),plt.title("Original")
plt.subplot(1,3,2),plt.imshow(F[:,:,0],'gray'),plt.title("Filtro  Paso Alto Butterworth")
plt.subplot(1,3,3),plt.imshow(IF,'gray'),plt.title("Imagen Seleccion")





### Ejemplo: Como detectar el emborronamiento 
El objetivo es usando la información frecuencial detectar si una imagen esta emborronada o no. Para ello se elimina la información de frecuencias bajas, p.ej aplicando un filtro paso alto ideal. Y obtenemos la energía media que queda. Si esta es menor que un determinado umbral entonces la imagen esta emborronada. 



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
def detect_emborronamiento_fft(image, size=60, thresh=10, vis=False):
    #Tomamos las dimensiones de la imagen
    
    (h, w) = image.shape
    (cX, cY) = (int(w / 2.0), int(h / 2.0))
    img_float32=np.float32(image)
    #aplicamos la transformada de fourier
    dft = cv2.dft(img_float32,flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift= np.fft.fftshift(dft)#centramos
    #si queremos visualizar la salida
    if vis:
        #Calculamos la magnitud del espectro
        magnitude = 20 * np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))
        #mostramos la imagen original
        (fig, ax) = plt.subplots(1, 2, )
        ax[0].imshow(image, cmap="gray")
        ax[0].set_title("Original")
        ax[0].set_xticks([])
        ax[0].set_yticks([])
        #mostramos la magnitud
        ax[1].imshow(magnitude, cmap="gray")
        ax[1].set_title("Magnitud")
        ax[1].set_xticks([])
        ax[1].set_yticks([])
        
    #Anulamos un recuadro de tamaño sizexsize desde el centro y reconstruimos
    dft_shift[cY - size:cY + size, cX - size:cX + size] = 0
    dft_shift = np.fft.ifftshift(dft_shift)
    recon = cv2.idft(dft_shift)   
    # Calculamos la magnitud de la imagen reconstruida
    # y la media  (otra forma de obtener la magnitud)
    
    magnitude = 20 * np.log(cv2.magnitude(recon[:,:,0],recon[:,:,1])) 
    mean = np.mean(magnitude)
    #La imagen esta emborronada si el valor medio de la mangnitude es 
    #menor que un umbral.
    return (mean, mean <= thresh)

#Con esta funcion pretenemos obtener un umbral
#para saber cuando la imagen esta emborronada
def getUmbral(i): 
    return 0


Vamos a probar el anterior código emborronando una imagen con una gausiana. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

gray = cv2.imread('miki_pluto.jpg',0)
width=450;
h,w=gray.shape;
factor = float(width/w)
hnew = int(h*factor)
gray = cv2.resize(gray, (hnew,width),cv2.INTER_AREA)
thr=240;

#Ejercicio estudiar formas de implementar la funcion getUmbral 
#thr = getUmbral(gray,0.75)

#Hacemos un bucle para obtener diferentes
#imagenes emborronadas
for radius in range(1, 30, 2):
    # clonamos la imagen original
    image = gray.copy()
    # Emborronamiento Gaussiano 
    image = cv2.GaussianBlur(image, (radius, radius),0)
    # apply our blur detector using the FFT
    (mean, blurry) = detect_emborronamiento_fft(image, size=60,thresh=thr)
    # escribimos en la imagen si esta emborronada o no y el promedio de energia
    #Creamos una imagen de color para escribir
    image = np.dstack([image] * 3) 
    color = (0, 0, 255) if blurry else (0, 255, 0)
    text = "Emborronada ({:.4f})" if blurry else "Sin Emborronamiento ({:.4f})"
    text = text.format(mean)
    cv2.putText(image, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,0.7, color, 2)
    print("[INFO] Kernel: {}, Resultado: {}".format(radius, text))
    # mostramos la imagen
    cv2.imshow("Imagen Test", image)
    cv2.waitKey(0)
cv2.destroyAllWindows()


## Transformada Coseno
A continuación se muestra un pequeño trozo de código para obtener la transformada coseno de una imagen.
La funcion dct() de OpenCV obtiene la transformada coseno. Para obtenerla debemos pasarle una imagen en niveles de grises con valores float. Dos flags son posibles en dct():
* cv2.DCT_INVERSE: La funcion hace la transformada directa 1D o 2D 
* cv2.DCT_ROWS: La funcion hace la transformada 1D a cada fila

cv2.dct(). This method takes a grayscale image in floating point. Pass flag cv2.DCT_INVERS

In [ ]:
import cv2

import numpy as np   

img =  cv2.imread('miki_pluto.jpg',0)

# convert the grayscale to float32
imf = np.float32(img) # float conversion

# find discrete cosine transform
#dst = cv2.dct(imf, cv2.DCT_INVERSE)
dst = cv2.dct(imf, cv2.DCT_ROWS)

# apply inverse discrete cosine transform
img1 = cv2.idct(dst)

# convert to uint8
img1 = np.uint8(img)

cv2.imshow("DCT", dst)
cv2.waitKey(0)
cv2.imshow("IDCT back image", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Ejercicios
<ol>
<li>Obtener una imagen $400\times 400$ que contenga un círculo blanco en el centro. Obtener la transformada de fourier.Mostrar el espectro y la fase. 
<li> Aplica el filtro paso bajo ideal, gausiano y Butterworth sobre la imagen dalmata.jpg. Explica que pasa cuando en el filtro paso bajo Butterworth el n cambia.     
<li>Aplica el filtro paso alto ideal, gausiano y Butterworth sobre la imagen dalmata.jpg. 
         
<li>Mejorar el código de la detección de emborronamiento de una imagen obteniendo una implementación de la función getUmbral. 
</ol>    
    